In [1]:
!pip install mediapipe opencv-python torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

def detect_and_landmark_face(image):
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.detections:
            bbox = results.detections[0].location_data.relative_bounding_box
            x, y, w, h = int(bbox.xmin * image.shape[1]), int(bbox.ymin * image.shape[0]), int(bbox.width * image.shape[1]), int(bbox.height * image.shape[0])
            face = image[y:y+h, x:x+w]
            return face
    return None

def extract_landmarks(face):
    with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True) as face_mesh:
        results = face_mesh.process(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks:
            landmarks = []
            for face_landmarks in results.multi_face_landmarks:
                for landmark in face_landmarks.landmark:
                    landmarks.append([landmark.x, landmark.y, landmark.z])
            return landmarks
    return None


In [3]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Define the MobileFaceNet model (Assume you have the MobileFaceNet model loaded)
class MobileFaceNet(torch.nn.Module):
    # Define the MobileFaceNet architecture here
    pass

# Load the pre-trained MobileFaceNet model
mobilefacenet_model = MobileFaceNet()
mobilefacenet_model.load_state_dict(torch.load('path_to_pretrained_model.pth'))  # Provide the correct path
mobilefacenet_model.eval()

def generate_embedding(face_image, model):
    preprocess = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

    face_image = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
    face_tensor = preprocess(face_image).unsqueeze(0)
    with torch.no_grad():
        embedding = model(face_tensor)
    return embedding.squeeze(0).cpu().numpy()


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_pretrained_model.pth'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def compare_embeddings(embedding1, embedding2):
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

# Example usage for real-time face authentication
cap = cv2.VideoCapture(0)
stored_embedding = None  # This should be pre-computed and stored

while True:
    ret, frame = cap.read()
    if not ret:
        break

    face = detect_and_landmark_face(frame)
    if face is not None:
        landmarks = extract_landmarks(face)
        if landmarks:
            # You can normalize or align the face here based on landmarks
            live_embedding = generate_embedding(face, mobilefacenet_model)
            if stored_embedding is not None:
                similarity = compare_embeddings(live_embedding, stored_embedding)
                print(f'Similarity: {similarity:.2f}')
                if similarity > 0.6:  # Adjust the threshold for your use case
                    cv2.putText(frame, 'Authenticated', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, 'Not Authenticated', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


MEDIAPIPE AND BUFFOLO_L MODEL

In [13]:
%pip install -U insightface mediapipe opencv-python torch torchvision


  Using cached insightface-0.7.3-cp311-cp311-win_amd64.whl
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached torch-2.4.0-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached torchvision-0.19.0-1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached onnx-1.16.2-cp311-cp311-win_amd64.whl.metadata (16 kB)
  Using cached Cython-3.0.11-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached albumentations-1.4.13-py3-none-any.whl.metadata (38 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached albucore-0.0.13-py3-none-any.whl.metadata (3.1 kB)
  Using cached eval_type_backport-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached torch-2.4.0-cp311-cp311-win_amd64.whl (197.9 MB

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\ASUS\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
%pip install onnxruntime

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
%pip install --upgrade insightface

  Using cached insightface-0.7.3-cp311-cp311-win_amd64.whl
  Using cached onnx-1.16.2-cp311-cp311-win_amd64.whl.metadata (16 kB)
  Using cached Cython-3.0.11-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached albumentations-1.4.13-py3-none-any.whl.metadata (38 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached albucore-0.0.13-py3-none-any.whl.metadata (3.1 kB)
  Using cached eval_type_backport-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached albumentations-1.4.13-py3-none-any.whl (171 kB)
Using cached Cython-3.0.11-cp311-cp311-win_amd64.whl (2.8 MB)
Using cached onnx-1.16.2-cp311-cp311-win_amd64.whl (14.4 MB)
Using cached albucore-0.0.13-py3-none-any.whl (8.5 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached pydantic-2.8.2-py3-none-any.w

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\ASUS\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:


import cv2
import mediapipe as mp
from insightface.app import FaceAnalysis  # Update the import path
from sklearn.metrics.pairwise import cosine_similarity

NameError: name 'framework' is not defined

In [4]:
import cv2
import mediapipe as mp
from insightface import app, FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity

ImportError: cannot import name 'FaceAnalysis' from 'insightface' (/usr/local/lib/python3.10/dist-packages/insightface/__init__.py)

In [14]:


# Initialize InsightFace buffalo_l model
model_pack_name = 'buffalo_l'
face_recognition_app = FaceAnalysis(name=model_pack_name)
face_recognition_app.prepare(ctx_id=0, det_size=(640, 640))

# Initialize Mediapipe for face detection and landmarking
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Function to compare embeddings
def compare_embeddings(embedding1, embedding2):
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

# Capture webcam video
cap = cv2.VideoCapture(0)
stored_embedding = None  # Assume you have a stored embedding for comparison

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Use Mediapipe to detect face and extract landmarks
    face = None
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.detections:
            bbox = results.detections[0].location_data.relative_bounding_box
            x, y, w, h = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])
            face = frame[y:y+h, x:x+w]

    if face is not None:
        # Use InsightFace buffalo_l model for recognition
        insight_faces = face_recognition_app.get(frame)
        if insight_faces:
            embedding_insightface = insight_faces[0]['embedding']  # Get the embedding from buffalo_l model

            if stored_embedding is not None:
                similarity_insight = compare_embeddings(embedding_insightface, stored_embedding)

                # Print or display the results
                print(f'InsightFace Similarity: {similarity_insight:.2f}')
                if similarity_insight > 0.6:  # Adjust the threshold as needed
                    cv2.putText(frame, 'Authenticated (InsightFace)', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, 'Not Authenticated', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'FaceAnalysis' is not defined

## AFTER CREATING ENVIRONMENT 

In [1]:
import cv2
import mediapipe as mp
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity

ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.

In [1]:
import mediapipe as mp


In [2]:
import insightface
from insightface.app import FaceAnalysis


ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.

In [7]:
import cv2
import mediapipe as mp
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity

ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.

In [6]:


# Initialize InsightFace buffalo_l model
model_pack_name = 'buffalo_l'
face_recognition_app = FaceAnalysis(name=model_pack_name)
face_recognition_app.prepare(ctx_id=0, det_size=(640, 640))

# Initialize Mediapipe for face detection and landmarking
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Function to compare embeddings
def compare_embeddings(embedding1, embedding2):
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

# Capture webcam video
cap = cv2.VideoCapture(0)
stored_embedding = None  # Assume you have a stored embedding for comparison

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Use Mediapipe to detect face and extract landmarks
    face = None
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.detections:
            bbox = results.detections[0].location_data.relative_bounding_box
            x, y, w, h = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])
            face = frame[y:y+h, x:x+w]

    if face is not None:
        # Use InsightFace buffalo_l model for recognition
        insight_faces = face_recognition_app.get(frame)
        if insight_faces:
            embedding_insightface = insight_faces[0]['embedding']  # Get the embedding from buffalo_l model

            if stored_embedding is not None:
                similarity_insight = compare_embeddings(embedding_insightface, stored_embedding)

                # Print or display the results
                print(f'InsightFace Similarity: {similarity_insight:.2f}')
                if similarity_insight > 0.6:  # Adjust the threshold as needed
                    cv2.putText(frame, 'Authenticated (InsightFace)', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                else:
                    cv2.putText(frame, 'Not Authenticated', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'FaceAnalysis' is not defined